# Make Dataset

features

In [1]:
#different make sequence function
import numpy as np
import pandas as pd
import os

robot_dof = 7

# Function to generate sequence names
def generate_sequence(base_name, indices):
    return [f'{base_name}{i}' for i in indices]

# Automate the sequence generation using the function
indices = range(robot_dof)  # Since the indices go from 0 to robot_dof

# Create sequences for tau, tau_d, tau_ext, q, q_d, dq, dq_d, e, de, and etau
tau = generate_sequence('tau_J', indices)
tau_d = generate_sequence('tau_J_d', indices)
tau_ext = generate_sequence('tau_ext', indices)

q = generate_sequence('q', indices)
q_d = generate_sequence('q_d', indices)

dq = generate_sequence('dq', indices)
dq_d = generate_sequence('dq_d', indices)

e = generate_sequence('e', indices)
de = generate_sequence('de', indices)
etau = generate_sequence('etau_J', indices)


In [2]:
def make_sequence(df, selected_features, seq_num,  gap):
    contact_indexs_ = df.loc[df.label.diff()>0.05,:].index
    contact_indexs2 = df.loc[df.label.diff()<-0.05,:].index
    contact_indexs = [idx for idx, idx2 in zip(contact_indexs_, contact_indexs2) if idx2 - idx >= seq_num]

    dataset_df = pd.DataFrame()
    for contact_index in contact_indexs:
        window_right_edge = contact_index
        end_point = contact_index + seq_num 
        for step in range(window_right_edge , end_point, gap):
            label = df.label[step-1]
            window = df[selected_features][step-seq_num:step]
            df_dummy=pd.DataFrame(np.insert(window.values.flatten(), 0, label).reshape(1,-1))
            dataset_df = pd.concat([dataset_df,df_dummy],ignore_index=True)
    return dataset_df

In [9]:
i

'dataset'

In [10]:
main_path = os.getcwd().replace('AIModels','')

#save_path = main_path+'/dataset/test_dataset_target_robot/'
#main_path = main_path + 'frankaRobot/DATA/dataset/target_robot/'

save_path = main_path+'dataset/test_dataset_source_robot/'
main_path = main_path + 'frankaRobot/DATA/dataset/source_robot/'

os.makedirs(save_path, exist_ok=True)

selected_features = e
seq_num = 28
gap = 5 # 25 ms

columns = range(seq_num*len(selected_features)+1)

df_master = pd.DataFrame(columns=columns)
for i in os.listdir(main_path):
    if len(i.split('.'))==1:
        file_path = main_path+i+'/labeled_data.csv'
        df = pd.read_csv(file_path)
        df.drop(columns='index', inplace=True)
        df = make_sequence(df, selected_features,seq_num, gap)
        #labeling data
        if 'link5' in i:
            df[0] = df[0]*5
        elif 'link6' in i:
            df[0] = df[0]*6
        elif 'f' in i:
            df[0] = df[0]*2
        elif 'e' in i:
            df[0] = df[0]*3
        #print(df.loc[:,0].min())
        df_master = df_master.append(df, ignore_index=True)

df_master.to_pickle(save_path+'dataset_test.pkl')
print(df_master.shape)


(378, 197)


In [15]:
df_master[0]

0       0.0
1       6.0
2       6.0
3       6.0
4       6.0
       ... 
1009    5.0
1010    5.0
1011    5.0
1012    5.0
1013    5.0
Name: 0, Length: 1014, dtype: float64

In [17]:
robot_dof = 6
main_path = os.getcwd().replace('AIModels','')


save_path = main_path+'/dataset/test_dataset_target_robot_ur5/'
main_path = main_path + '/urRobot/DATA/dataset/UR5/'
'''
save_path = main_path + '/dataset/test_dataset_target_robot_ur10/'
main_path = main_path + '/urRobot/DATA/dataset/UR10/'
'''
# Function to generate sequence names
def generate_sequence(base_name, indices):
    return [f'{base_name}{i}' for i in indices]

# Automate the sequence generation using the function
indices = range(robot_dof)  # Since the indices go from 0 to robot_dof
selected_features = generate_sequence('e', indices)

os.makedirs(save_path, exist_ok=True)
seq_num = 28
gap = 5 # 25 ms

columns = range(seq_num*len(selected_features)+1)

df_master = pd.DataFrame(columns=columns)
for i in os.listdir(main_path):
    if len(i.split('.'))==1:
        file_path = main_path+i+'/labeled_data.csv'
        df = pd.read_csv(file_path)
        df = make_sequence(df, selected_features,seq_num, gap)
        #labeling data
        if 'link5' in i:
            df[0] = df[0]*5
        elif 'link6' in i:
            df[0] = df[0]*6
        elif 'link3' in i:
            df[0] = df[0]*3
        elif 'link4' in i:
            df[0] = df[0]*4
        #print(df.loc[:,0].min())
        df_master = df_master.append(df, ignore_index=True)

df_master.to_pickle(save_path+'dataset_test.pkl')
print(df_master.shape)


(1950, 169)


In [18]:
df_master.head()

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,167,168
0,0.0,0.000235,-6.200000e-05,0.000132,0.000009,-0.000007,-0.000015,0.000220,-0.000069,0.000138,...,0.000405,-0.000047,-0.000005,-0.000098,0.000088,0.000123,0.000389,-0.000045,-0.000008,-0.000117
1,3.0,0.000189,-3.400000e-05,0.000242,-0.000051,0.000007,-0.000067,0.000168,-0.000036,0.000250,...,0.000474,-0.000057,-0.000011,-0.000094,0.000009,0.000152,0.000457,-0.000036,-0.000014,-0.000080
2,3.0,0.000163,1.000000e-06,0.000320,-0.000031,-0.000053,-0.000114,0.000184,-0.000006,0.000344,...,0.000493,-0.000026,0.000026,-0.000044,0.000023,0.000196,0.000480,-0.000013,-0.000017,0.000008
3,3.0,0.000175,3.500000e-05,0.000401,-0.000016,0.000019,-0.000177,0.000160,0.000049,0.000410,...,0.000470,-0.000019,-0.000030,0.000044,-0.000072,0.000258,0.000486,-0.000015,-0.000018,0.000022
4,3.0,0.000164,8.000000e-05,0.000434,-0.000025,-0.000012,-0.000154,0.000143,0.000050,0.000406,...,0.000503,-0.000005,-0.000044,0.000077,-0.000097,0.000258,0.000490,0.000000,-0.000028,0.000091
